# DeepMind_NLP-3

## Language models
Objective: Build a language model assings a probability to a sequence of words, such that $\sum_{w\in \sum{*}}p(w) = 1$ 

* \* : any string

Given the observed training text, how probable is this new utterance(말)?

Thus we can compare different orderings of words.(e.g. Translation)

P(he likes apples) > p(apples likes he)

or choice of words(e.g. Speech Recognitions)

P(he likes apples) > P(he licks apples)

Much of NLP can be structured as (conditional) language modelling:
* Translation:
* QA
* Dialogue

Most language models employ the chain rule to decompose the joint probability into a sequence of conditional probabilities:

$P(w_1, w_2, \cdots, w_N) = p(w_1)P(w_2|w_1)P(w_3|w_1,w_2)\times \cdots \times P(w_N|w_1, w_2, \cdots ,w_{N-1})$

The simple objective of modelling the next word given the ovserved history contains much of the complexity of natural language understanding. 다음 단어를 예측하는 것은 과거 문장이해, 복잡도가 얼만큼 관찰 되었는가 연관

Consider predicting the extension of the utterance:

P(?|There she built a)

With more context, able to use knowledge of both language and the world to heavily constrain the distribution over the next word. context가 많을 수록 문제(확률)의 범위를 좁힘

P(?| Alice went toe the beach. There she built a)

There is evidence that human language acquisition partly relies on future prediciton

## Evaluating a Language Model
A good model assigns real utterances $w_1^N$ from a language a high probability. 

This can be measured with cross entropy:

$$H(w_1^N) = −\dfrac{1}{N} \log_2 P(w_1^N)$$

Intuition 1: Cross entropy is a measure of how many bits are needed to encode text with our model.

Alternatively we can use **perplexity**: 
$$perplexity(w_1^N ) = 2^{H(w_1^N)}$$
Intuition 2: Perplexity is a measure of how surprised our model is on seeing each word.

Language modelling is a time series prediction problem in which we must be careful to train on the past and test on the future. 과거의 경험으로 미래를 예측, 훈련 / 시험 데이터를 잘 나눌 것, mixed up 하지 말것 

If the corpus is composed of articles, it is best to ensure the test data is drawn from a disjoint set of articles to the training data.

* 나쁜 예: 반쪽 짜리 소설을 훈련데이터에, 나머지 반쪽 짜리 소설을 시험 데이터에 
* 많은 데이터가 필요함, billions of words로 해야함


Two popular data sets for language modelling evaluation are a preprocessed version of the Penn [Treebank](www.fit.vutbr.cz/~imikolov/rnnlm/simple- examples.tgz), and the [Billion Word Corpus]( code.google.com/p/1- billion- word- language- modeling- benchmark/) Both are flawed:
* the PTB is very small and has been heavily processed. As such it is not representative of natural language.
* The Billion Word corpus was extracted by first randomly permuting sentences in news articles and then splitting into training and test sets. As such train and test sentences come from the same articles and overlap in time.

The recently introduced WikiText datasets(Pointer Sentinel Mixture Models. Merity et al., arXiv 2016) are a better option.

## Lecture Overview
The rest of this lecture will survey three approaches to parametrising language models:
* With count based n-gram models we approximate the history of observed words with just the previous n words.
* Neural n-gram models embed the same fixed n-gram history in a continues space and thus better capture correlations between histories.
* With Recurrent Neural Networks we drop the fixed n-gram history and compress the entire history in a fixed length vector, enabling long range correlations to be captured.

### Count based N-Gram Language Models

#### N-Gram Models: The Markov Chain Assumption
Markov assumption:
* only previous history matters
* limited memory: only last k − 1 words are included in history (older words less relevant)
* k-th order Markov model
For instance 2-gram language model:
$\begin{eqnarray} p(w_1,w_2,w_3, \cdots ,w_n)
& = & p(w_1) p(w_2|w_1) p(w_3|w_1, w_2) \times \cdots \times p(w_n|w_1, w_2, . . . w_{n−1}) \newline
& \approx & p(w_1) p(w_2|w_1) p(w_3|w_2)\times \cdots \times p(w_n|w_{n−1})
\end{eqnarray}$

The conditioning context, $w_{i−1}$, is called the history.

#### N-Gram Models: Estimating Probabilities
Maximum likelihood estimation for 3-grams: 
$p(w_3|w_1, w_2) = \dfrac{count(w_1, w_2, w_3)}{count(w_1, w_2)}$

Collect counts over a large text corpus. Billions to trillions of words are easily available by scraping the web.

#### N-Gram Models: Back-Off
Maximum Likelihood is not a great estimator for this sort of problems

In our training corpus we may never observe the trigrams:
* Oxford Pimm’s eater
* Oxford Pimm’s drinker
If both have count 0 our smoothing methods will assign the same probability to them.
A better solution is to interpolate with the bigram probability:
* Pimm’s eater
* Pimm’s drinker

이전 두 단어(trigram)을 보고도 다음 단어를 잘 모르겠다 싶으면, smoothing 해서 이전 한 단어(bigram)을 보고 다음 단어를 예측는 방법(hierarchical smoothing)

By recursively interpolating the n-gram probabilities with the (n − 1)-gram probabilities we can smooth our language model and ensure all words have non-zero probability in a given context.

A simple approach is linear interpolation: (각종 gram을 가중 합을 하는것)

$p_I (w_n|w_{n−2}, w_{n−1}) = \lambda_3p(w_n|w_{n−2}, w_{n−1}) + \lambda_2p(w_n|w_{n−1}) + \lambda_1p(w_n)$

where $\lambda_3 + \lambda_2 + \lambda_1 = 1$.

A number of more advanced smoothing and interpolation schemes have been proposed, with Kneser-Ney being the most common. 
* (**나중에 한번 보기**) Smoothing 관련 기술들
    * An empirical study of smoothing techniques for language modeling. Stanley Chen and Joshua Goodman. Harvard University, 1998. [링크](http://aclweb.org/anthology/P96-1041)   
    * the art of the smoothing language model = try and get the posterior distribution(사후확률 분포: posterior probability $\propto$ likelihood $\times$ prior) from the language model to match that which we see in real language
    * the real language is that we see power laws in the frequency of words (called **[heaps law](https://en.wikipedia.org/wiki/Heaps%27_law)**: 말뭉치(corpus)를 늘려갈 수록 단어(unique한 토큰)의 갯수가 로그함수 적으로 올라간다는 법칙).
    * rule-base 가 language model에 적합하지 않는 이유도 여기에 있다. 계속 새로운 단어들이 나옴

#### Provisional Summary

Good
* Count based n-gram models are exceptionally scalable and are able to be trained on trillions of words of data,
* fast constant time evaluation of probabilities at test time,
* sophisticated smoothing techniques match the empirical distribution
of language.

Bad
* Large ngrams are sparse, so hard to capture long dependencies,
* symbolic nature does not capture correlations between semantically
similary word distributions, e.g. cat ↔ dog,
* similarly morphological regularities(형태학적 규칙을 못찾음), running ↔ jumping, or gender.

### Neural N-Gram Language Models
Replace the multinomial distribustions with Simple Feed forward network

 <img src="./figs/trigramNN.png" alt="Drawing" style="width: 300px;"/>

 <img src="./figs/Sampling1.png" alt="Drawing" style="width: 500px;"/>

 <img src="./figs/Sampling2.png" alt="Drawing" style="width: 500px;"/>

 <img src="./figs/Backpropogation.png" alt="Drawing" style="width: 300px;"/>

 <img src="./figs/Backpropogation2.png" alt="Drawing" style="width: 500px;"/>